In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import scipy
import sklearn
from sklearn import preprocessing
from scipy.io import loadmat
from numpy import linalg as LA
import mkl_algorithms
import graphgeneration
from graphgeneration import graphgeneration
from mkl_algorithms import OMKR
from mkl_algorithms import RBF
from mkl_algorithms import POLY
from mkl_algorithms import RFOMKR
from mkl_algorithms import Raker
from mkl_algorithms import OMKLGF
from mkl_algorithms import OMKLSFG

lam = .001
data = np.genfromtxt('Concretedata.txt',skip_header=1,encoding='utf-8')
X = data[:,:-1] 
Y = data[:,-1:]
M, N = X.shape
Y = (Y/(np.max(Y)-np.min(Y))) - np.min(Y)/(np.max(Y)-np.min(Y))
X_n = np.zeros((1,M))
for i in range(0,M):
    X_n[0,i] = LA.norm(X[i,:])
X = X/np.max(X_n)

np.random.seed(seed=0)
gamma = []
for i in range(0,51):
    gamma.append(10**(4*(i/50)-2))
gamma = np.array(gamma)


mse_omkr = np.zeros((M,1))
mse_rbf1 = np.zeros((M,1))
mse_poly2 = np.zeros((M,1))
time_rfomkr = np.zeros((1,20))
mse_rfomkr = np.zeros((M,1))
time_raker = np.zeros((1,20))
mse_raker = np.zeros((M,1))
time_omklgf1 = np.zeros((1,20))
mse_omklgf1 = np.zeros((M,1))
time_omklgf10 = np.zeros((1,20))
mse_omklgf10 = np.zeros((M,1))
time_omklgf20 = np.zeros((1,20))
mse_omklgf20 = np.zeros((M,1))
time_omkldom = np.zeros((1,20))
mse_omkldom = np.zeros((M,1))
time_omkldom10 = np.zeros((1,20))
mse_omkldom10 = np.zeros((M,1))
time_omkldom20 = np.zeros((1,20))
mse_omkldom20 = np.zeros((M,1))
time_omkldomr = np.zeros((1,20))
mse_omkldomr = np.zeros((M,1))
time_omkldomr10 = np.zeros((1,20))
mse_omkldomr10 = np.zeros((M,1))
time_omkldomr20 = np.zeros((1,20))
mse_omkldomr20 = np.zeros((M,1))
n_components = 50

# OMKR
e = np.zeros((M,1))
w = (1/gamma.shape[0])*np.ones((1,gamma.shape[0]))
theta = np.zeros((gamma.shape[0],1))
start_time = time.time()
for i in range(0,M):
    eta = .1/np.sqrt(i+1)
    omkr = OMKR(eta, gamma)
    f_RF, f_RF_p = omkr.predict(X[0:i+1,:], w, theta)
    w, theta = omkr.update(f_RF_p, Y[i], theta, w)
    if i==0:
        e[0] = (f_RF-Y[0])**2
    else:
        e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
end_time = time.time()
time_omkr = end_time-start_time
mse_omkr = e

# RBF-1
e = np.zeros((M,1))
gamma1 = []
gamma1.append(1)
theta = np.zeros((1,1))
start_time = time.time()
for i in range(0,M):
    eta = .1/np.sqrt(i+1)
    rbf = RBF(eta, gamma1)
    f_RF = rbf.predict(X[0:i+1,:], theta)
    theta = rbf.update(f_RF, Y[i], theta)
    if i==0:
        e[0] = (f_RF-Y[0])**2
    else:
        e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
end_time = time.time()
time_rbf1 = end_time-start_time
mse_rbf1 = e

# POLY-2
e = np.zeros((M,1))
gamma1 = []
gamma1.append(2)
theta = np.zeros((1,1))
start_time = time.time()
for i in range(0,M):
    eta = .1/np.sqrt(i+1)
    poly = POLY(eta, gamma1)
    f_RF = poly.predict(X[0:i+1,:], theta)
    theta = poly.update(f_RF, Y[i], theta)
    if i==0:
        e[0] = (f_RF-Y[0])**2
    else:
        e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
end_time = time.time()
time_poly2 = end_time-start_time
mse_poly2 = e

for cc in range(0,20):
    # Generating Random Features
    rbf_feature = np.zeros((N,n_components,gamma.shape[0]))
    for i in range(0,gamma.shape[0]):
        rbf_feature[:,:,i] = np.random.randn(N,n_components)*np.sqrt(1/gamma[i])
    
    # RFOMKR
    e = np.zeros((M,1))
    w = (1/gamma.shape[0])*np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        rfomkr = RFOMKR(rbf_feature, eta)
        f_RF, f_RF_p, X_features = rfomkr.predict(X[i:i+1,:], theta, w, )
        w, theta = rfomkr.update(f_RF_p, Y[i], theta, w, X_features)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_rfomkr[cc:cc+1] = end_time-start_time
    mse_rfomkr = ( 1/(cc+1) )*( (cc*mse_rfomkr)+e )
    
    # Raker
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        raker = Raker(lam, rbf_feature, eta)
        f_RF, f_RF_p, X_features = raker.predict(X[i:i+1,:], theta, w)
        w, theta = raker.update(f_RF_p, Y[i], theta, w, X_features)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_raker[cc:cc+1] = end_time-start_time
    mse_raker = ( 1/(cc+1) )*( (cc*mse_raker)+e )
    
    # OMKL-GF with M = 10 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 10
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, p_kk = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n, p_c, I_t = omklgf.predict(X[i:i+1,:], theta, w, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, p_kk, p_c, ter)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if i>500:
            ter = 1
    end_time = time.time()
    time_omklgf10[cc:cc+1] = end_time-start_time
    mse_omklgf10 = ( 1/(cc+1) )*( (cc*mse_omklgf10)+e )
    
    # OMKL-GF with M = 1 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 1
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, p_kk = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n, p_c, I_t = omklgf.predict(X[i:i+1,:], theta, w, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, p_kk, p_c, ter)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if i>500:
            ter = 1
    end_time = time.time()
    time_omklgf1[cc:cc+1] = end_time-start_time
    mse_omklgf1 = ( 1/(cc+1) )*( (cc*mse_omklgf1)+e )
    
    # OMKL-GF with M = 20 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 20
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, p_kk = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n, p_c, I_t = omklgf.predict(X[i:i+1,:], theta, w, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, p_kk, p_c, ter)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if i>500:
            ter = 1
    end_time = time.time()
    time_omklgf20[cc:cc+1] = end_time-start_time
    mse_omklgf20 = ( 1/(cc+1) )*( (cc*mse_omklgf20)+e )
    
    #OMKL-SFG with M=10
    e = np.zeros((M,1))
    m = 10
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    d_set = omkldom.greedysetcover()
    s_t = 0
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict(X[i:i+1,:], theta, w, z, eta, d_set, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldom10[cc:cc+1] = end_time-start_time
    mse_omkldom10 = ( 1/(cc+1) )*( (cc*mse_omkldom10)+e )
    
    #OMKL-SFG with M=1
    e = np.zeros((M,1))
    m = 1
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    d_set = omkldom.greedysetcover()
    s_t = 0
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict(X[i:i+1,:], theta, w, z, eta, d_set, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldom[cc:cc+1] = end_time-start_time
    mse_omkldom = ( 1/(cc+1) )*( (cc*mse_omkldom)+e )
    
    #OMKL-SFG with M=20
    e = np.zeros((M,1))
    m = 20
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    d_set = omkldom.greedysetcover()
    s_t = 0
    for i in range(0,M):
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict(X[i:i+1,:], theta, w, z, eta, d_set, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldom20[cc:cc+1] = end_time-start_time
    mse_omkldom20 = ( 1/(cc+1) )*( (cc*mse_omkldom20)+e )
    
    #OMKL-SFG-R with M=10
    e = np.zeros((M,1))
    m = 10
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    s_t = 0
    for i in range(0,M):
        z_bar = z/np.sum(z)
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        if ter==0:
            u_bar = np.sort(z_bar)
            ep = (1-eta)*u_bar[0,-10] + (eta/gamma.shape[0]) - 1e-7
            d_set_ref, B_t, n_ind_ref, gamma_ind_ref = omkldom.graphrefinement(ep,z,eta)
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict_refinement(X[i:i+1,:], theta, w, z, eta, d_set_ref,\
                                                                                  B_t,n_ind_ref, gamma_ind_ref, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldomr10[cc:cc+1] = end_time-start_time
    mse_omkldomr10 = ( 1/(cc+1) )*( (cc*mse_omkldomr10)+e )
    
    #OMKL-SFG-R with M=1
    e = np.zeros((M,1))
    m = 1
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    s_t = 0
    for i in range(0,M):
        z_bar = z/np.sum(z)
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        if ter==0:
            u_bar = np.sort(z_bar)
            ep = (1-eta)*u_bar[0,-10] + (eta/gamma.shape[0]) - 1e-7
            d_set_ref, B_t, n_ind_ref, gamma_ind_ref = omkldom.graphrefinement(ep,z,eta)
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict_refinement(X[i:i+1,:], theta, w, z, eta, d_set_ref,\
                                                                                  B_t,n_ind_ref, gamma_ind_ref, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldomr[cc:cc+1] = end_time-start_time
    mse_omkldomr = ( 1/(cc+1) )*( (cc*mse_omkldomr)+e )
    
    #OMKL-SFG-R with M=20
    e = np.zeros((M,1))
    m = 20
    ter = 0
    c_n = 0
    w = np.ones((1,gamma.shape[0]))
    z = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    A_t, KL, gamma_ind, n_ind = graphgeneration(gamma, m)
    omkldom = OMKLSFG(lam, rbf_feature, gamma, A_t, KL, gamma_ind, n_ind)
    s_t = 0
    for i in range(0,M):
        z_bar = z/np.sum(z)
        eta = .1/np.sqrt(i+1)
        if i>=500:
            ter = 1
        if ter==0:
            u_bar = np.sort(z_bar)
            ep = (1-eta)*u_bar[0,-10] + (eta/gamma.shape[0]) - 1e-7
            d_set_ref, B_t, n_ind_ref, gamma_ind_ref = omkldom.graphrefinement(ep,z,eta)
        f_RF, f_RF_p, X_features, n_n, s_n, p, q = omkldom.predict_refinement(X[i:i+1,:], theta, w, z, eta, d_set_ref,\
                                                                                  B_t,n_ind_ref, gamma_ind_ref, ter, s_t)
        w, z, theta = omkldom.update(f_RF, f_RF_p, Y[i], theta, w, z, X_features, n_n, s_n, p, q, eta)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkldomr20[cc:cc+1] = end_time-start_time
    mse_omkldomr20 = ( 1/(cc+1) )*( (cc*mse_omkldomr20)+e )


print('MSE of OMKR is %s' %mse_omkr[-1])
print('Execution time of OMKR is %s' %time_omkr)
print('MSE of RBF-1 is %s' %mse_rbf1[-1])
print('Execution time of RBF-1 is %s' %time_rbf1)
print('MSE of POLY-2 is %s' %mse_poly2[-1])
print('Execution time of POLY-2 is %s' %time_poly2)
print('MSE of Raker is %s' %mse_raker[-1])
print('Execution time of Raker is %s' %np.mean(time_raker))
print('MSE of RFOMKR is %s' %mse_rfomkr[-1])
print('Execution time of RFOMKR is %s' %np.mean(time_rfomkr))
print('MSE of OMKL-GF with M=1 is %s' %mse_omklgf1[-1])
print('Execution time of OMKL-GF with M=1 is %s' %(np.mean(time_omklgf1)))
print('MSE of OMKL-GF with M=10 is %s' %mse_omklgf10[-1])
print('Execution time of OMKL-GF with M=10 is %s' %(np.mean(time_omklgf10)))
print('MSE of OMKL-GF with M=20 is %s' %mse_omklgf20[-1])
print('Execution time of OMKL-GF with M=20 is %s' %(np.mean(time_omklgf20)))
print('MSE of OMKL-SFG with M=1 is %s' %mse_omkldom[-1])
print('Execution time of OMKL-SFG with M=1 is %s' %np.mean(time_omkldom))
print('MSE of OMKL-SFG with M=10 is %s' %mse_omkldom10[-1])
print('Execution time of OMKL-SFG with M=10 is %s' %np.mean(time_omkldom10))
print('MSE of OMKL-SFG with M=20 is %s' %mse_omkldom20[-1])
print('Execution time of OMKL-SFG with M=20 is %s' %np.mean(time_omkldom20))
print('MSE of OMKL-SFG-R with M=1 is %s' %mse_omkldomr[-1])
print('Execution time of OMKL-SFG-R with M=1 is %s' %np.mean(time_omkldomr))
print('MSE of OMKL-SFG-R with M=10 is %s' %mse_omkldomr10[-1])
print('Execution time of OMKL-SFG-R with M=10 is %s' %np.mean(time_omkldomr10))
print('MSE of OMKL-SFG-R with M=20 is %s' %mse_omkldomr20[-1])
print('Execution time of OMKL-SFG-R with M=20 is %s' %np.mean(time_omkldomr20))

plt.semilogy(mse_omkr,'--',linewidth=3)
plt.semilogy(mse_raker,'--',linewidth=3)
plt.semilogy(mse_omklgf10,linewidth=3)
plt.semilogy(mse_omkldom10,linewidth=3)
plt.semilogy(mse_omkldomr10,linewidth=3)
plt.xlabel('Time Index')
plt.ylabel('MSE')
plt.legend(['OMKR', 'Raker', 'OMKL-GF', 'OMKL-SFG', 'OMKL-SFG-R'])